In [28]:
import pandas as pd
import json
import requests
import numpy as np

# Points intersecction

Procedure definition:


```sql

DROP FUNCTION get_aqpoints(weights_raw TEXT, points_array TEXT); 

DROP TYPE weight_intersection;

CREATE TYPE weight_intersection AS (points_id numeric, basinid numeric, the_geom geometry, water_risk numeric, water_stress numeric, interannual_variability numeric, seasonal_variability numeric, flood_occurrence numeric, drought_severity numeric, upstream_storage numeric, groundwater_stress numeric, return_flow_ratio numeric, upstream_protected_land numeric, media_coverage numeric, access_to_water numeric, threatened_amphibians numeric);

CREATE OR REPLACE FUNCTION get_aqpoints(weights_raw TEXT, points_array TEXT) 
	RETURNS SETOF weight_intersection as $$
	DECLARE
    query1 TEXT;
    weights TEXT;
    weights_sum TEXT;
	BEGIN
        query1:='select array_to_string(array_agg(col::text ||''*''|| weights::text), ''+ '') as weights, sum(weights)::text as weights_sum from (select unnest(Array'|| weights_raw||') as weights, unnest(array[''bws_s'', ''wsv_s'', ''sv_s'', ''hfo_s'', ''dro_s'', ''stor_s'', ''gw_s'', ''wri_s'', ''eco_s_s'', ''mc_s'', ''wcg_s'', ''eco_v_s'']) as col) as tables where weights is not null';
        
        EXECUTE query1 into weights, weights_sum;
        
		RETURN query EXECUTE 'with points as (SELECT st_setsrid(st_geomfromtext(points),4326) as the_geom, row_number() over () as points_id FROM unnest(Array'|| points_array ||') as points), ranges as (select basinid, average, min(average) over (), max(average) over (), bws_s, wsv_s, sv_s, hfo_s, dro_s, stor_s, gw_s, wri_s, eco_s_s, mc_s, wcg_s, eco_v_s from (SELECT basinid, ('|| weights ||')/('|| weights_sum ||') as average, bws_s, wsv_s, sv_s, hfo_s, dro_s, stor_s, gw_s, wri_s, eco_s_s, mc_s, wcg_s, eco_v_s FROM water_risk_weights) initial) select points_id::numeric, ranges.basinid::numeric, points.the_geom, (((average-min)/(max-min))*5)::numeric as water_risk, bws_s::numeric as baseline_water_stress, wsv_s::numeric as interannual_variability, sv_s::numeric as seasonal_variability, hfo_s::numeric as flood_occurrence, dro_s::numeric as drought_severity, stor_s::numeric as upstream_storage, gw_s::numeric as groundwater_stress, wri_s::numeric as return_flow_ratio, eco_s_s::numeric as upstream_protected_land, mc_s::numeric as media_coverage, wcg_s::numeric as access_to_water, eco_v_s::numeric as threatened_amphibians from ranges inner join wri_subcatchements on ranges.basinid=wri_subcatchements.basinid right join points on  st_intersects(wri_subcatchements.the_geom, points.the_geom)';
	
	END
$$ language 'plpgsql';

```



Sql Example:

```sql
Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]','[''POINT(84.8085584935 -14.20067639)'', ''POINT(54.0392656274 -70.8898132233)'', ''POINT(-28.5832686897 -7.71134965117)'', ''POINT(47.9458596199 82.5599787066)'', ''POINT(50.8126903314 -46.0154993389)'', ''POINT(-62.5229253542 -9.68983337791)'', ''POINT(-34.7977234627 84.9984574252)'', ''POINT(-80.1102876685 -33.9286081419)'', ''POINT(22.6686500117 -85.6713992254)'', ''POINT(44.359802466 -27.7294728889)'']')
```



Sql Template:

``` sql
Select * from get_aqpoints({weights_scheeme},{points array})

```





In [41]:
payload = {'q': "Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]','[''POINT(84.8085584935 -14.20067639)'', ''POINT(54.0392656274 -70.8898132233)'', ''POINT(-28.5832686897 -7.71134965117)'', ''POINT(47.9458596199 82.5599787066)'', ''POINT(50.8126903314 -46.0154993389)'', ''POINT(-62.5229253542 -9.68983337791)'', ''POINT(-34.7977234627 84.9984574252)'', ''POINT(-80.1102876685 -33.9286081419)'', ''POINT(22.6686500117 -85.6713992254)'', ''POINT(44.359802466 -27.7294728889)'']')"}
r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(0)

,access_to_water,basinid,drought_severity,flood_occurrence,groundwater_stress,interannual_variability,media_coverage,points_id,return_flow_ratio,seasonal_variability,the_geom,threatened_amphibians,upstream_protected_land,upstream_storage,water_risk,water_stress


# Anexo: Points stress test

test points; 100; 500; 1000; 100000;

In [50]:
t = 180 * np.random.rand(1000000,2) - 90

'POINT(-72.0 42.2)'

In [51]:
example1=[]
for point in t:
    example1.append('\'\'POINT(' +str(point[0])+ ' ' + str(point[1]) +')\'\'')

In [55]:
t10 = '['+', '.join(example1[:10])+']'
t100 = '['+', '.join(example1[:100])+']'
t1000 = '['+', '.join(example1[:1000])+']'
t10000 = '['+', '.join(example1[:10000])+']'
print(t10)
print(len(t10))
print(len(t100))
print(len(t1000))
print(len(t10000))

[''POINT(59.5811260967 77.3623262486)'', ''POINT(-63.6218410705 89.1303836085)'', ''POINT(48.0451669995 -80.5556332888)'', ''POINT(-57.0569892769 -74.2317807807)'', ''POINT(-46.3315737453 13.6809865413)'', ''POINT(8.24187341096 49.2353479855)'', ''POINT(70.4610676554 -32.7375542753)'', ''POINT(47.154345797 -77.3688758508)'', ''POINT(-60.7940368148 -42.7692197099)'', ''POINT(-86.1714505549 -47.6013917246)'']
410
4097
40771
408083


In [59]:
payload = {'q': ""}
payload['q']="Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]',\'"+ t10 +"\')"

r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)
tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(0)

,access_to_water,basinid,drought_severity,flood_occurrence,groundwater_stress,interannual_variability,media_coverage,points_id,return_flow_ratio,seasonal_variability,the_geom,threatened_amphibians,upstream_protected_land,upstream_storage,water_risk,water_stress


In [63]:
payload = {'q': ""}
payload['q']="Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]',\'"+ t100 +"\')"
r = requests.get('https://wri-01.carto.com/api/v2/sql', params=payload)

tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(0)

<Response [200]>


,access_to_water,basinid,drought_severity,flood_occurrence,groundwater_stress,interannual_variability,media_coverage,points_id,return_flow_ratio,seasonal_variability,the_geom,threatened_amphibians,upstream_protected_land,upstream_storage,water_risk,water_stress


In [80]:
payload = {'q': ""}
payload['q']="Select * from get_aqpoints('[4, 1, 0.5, 1, 1, 2, 2, 1, 0.5, 1, 2, 0.5]',\'"+ t10000 +"\')"
r = requests.post('https://wri-01.carto.com/api/v2/sql', data=payload)
if r.status_code != 200:
    issue = json.loads(r.text)
    print(issue[u'title'])
    print(issue[u'comments'])    
else:
    tableStructure= pd.read_json(json.dumps(r.json()['rows']), orient='records')
tableStructure.head(10)
        



,access_to_water,basinid,drought_severity,flood_occurrence,groundwater_stress,interannual_variability,media_coverage,points_id,return_flow_ratio,seasonal_variability,the_geom,threatened_amphibians,upstream_protected_land,upstream_storage,water_risk,water_stress
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0101000020E6100000DD16065762CA4D400A0E6F5A3057...,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,0101000020E6100000B68DFA7C98CF4FC0499C7D345848...,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,0101000020E610000000DB4008C8054840C1FDEC7E8F23...,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,0101000020E61000000C41B46C4B874CC0AA3C0E7FD58E...,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,0101000020E61000004B232C02712A47C09FCC9744AA5C...,NaN,NaN,NaN,NaN,NaN
5,0.0,9007.0,2.009725,3.578902,0.000000,0.862708,0.000000,6,1.874997,0.986695,0101000020E6100000BAB0EBD4D67B20407473FEE11F9E...,1.0,3.18696,5.0,2.101333,1.706493
6,0.0,9007.0,2.009725,3.578902,0.000000,0.862708,1.896907,6,1.874997,0.986695,0101000020E6100000BAB0EBD4D67B20407473FEE11F9E...,1.0,3.18696,5.0,2.245030,1.706493
7,0.0,9007.0,2.009725,3.578902,0.000000,0.862708,0.000000,6,1.874997,0.986695,0101000020E6100000BAB0EBD4D67B20407473FEE11F9E...,1.0,3.18696,5.0,2.101333,1.706493
8,0.0,9007.0,2.009725,3.578902,0.482125,0.862708,1.896907,6,1.874997,0.986695,0101000020E6100000BAB0EBD4D67B20407473FEE11F9E...,1.0,3.18696,5.0,2.318075,1.706493
9,0.0,9007.0,2.009725,3.578902,0.635615,0.862708,0.000000,6,1.874997,0.986695,0101000020E6100000BAB0EBD4D67B20407473FEE11F9E...,1.0,3.18696,5.0,2.197633,1.706493
